In [0]:
# UDF's

In [0]:
from pyspark.sql import functions as F, types as T

In [0]:
data1 = [
    (100, "dileep kumar"),
    (101, "rajeev kanakala"),
    (102, "ganesh yadav"),
    (103, "lokesh reddy")
]

columns = T.StructType([
    T.StructField("EmpId", T.IntegerType(), True),
    T.StructField("EmpFullName", T.StringType(), True)
])

df = spark.createDataFrame(data=data1, schema=columns)
df.display()

In [0]:
"dileep".upper()

In [0]:
def rename_and_cast(name):
    full_name = name.split(" ")   # ---> list
    first_name = full_name[0]
    last_name = full_name[1]
    first_name = first_name[0].upper() + first_name[1:]
    last_name = last_name[0].upper() + last_name[1:]
    return f"{first_name} {last_name}"

In [0]:
rename_and_cast("dileep kumar")

In [0]:
# we need to register the python function as a udf
# Method - 1

udf_rename_and_cast = F.udf(f=rename_and_cast, returnType=T.StringType())

In [0]:
type(udf_rename_and_cast)

In [0]:
df = (
    df
    .withColumn("EmpFullName", udf_rename_and_cast(F.col("EmpFullName")))
)

df.display()

In [0]:
renamed_df = (
    df
    .withColumns(
        {
            "EmpFullName": udf_rename_and_cast(F.col("EmpFullName")),
            "EmpFirstName": F.split(F.col("EmpFullName"), " ")[0],
            "EmpLastName": F.split(F.col("EmpFullName"), " ")[1]
        }
    )
    .drop("EmpFullName")
)

renamed_df.display()

In [0]:
renamed_df = (
    df
    .withColumn("EmpFullName", udf_rename_and_cast(F.col("EmpFullName")))
    .withColumn("EmpFirstName", F.split(F.col("EmpFullName"), " ")[0])
    .withColumn("EmpLastName", F.split(F.col("EmpFullName"), " ")[1])
    .drop("EmpFullName")
)

renamed_df.display()

In [0]:
# Method - 2

@udf(returnType=T.StringType())
def rename_and_cast2(name):
    full_name = name.split(" ")   # ---> list
    first_name = full_name[0]
    last_name = full_name[1]
    first_name = first_name[0].upper() + first_name[1:]
    last_name = last_name[0].upper() + last_name[1:]
    return f"{first_name} {last_name}"

In [0]:
renamed_df = (
    df
    .withColumn("EmpFullName", rename_and_cast2(F.col("EmpFullName")))
    .withColumn("EmpFirstName", F.split(F.col("EmpFullName"), " ")[0])
    .withColumn("EmpLastName", F.split(F.col("EmpFullName"), " ")[1])
    .drop("EmpFullName")
)

renamed_df.display()